In [1]:
from lxml import html
import pandas as pd
import requests
import re

In [2]:
venue = 'wsdm'
year  = '2012'
vtype = 'conf'

In [3]:
def build_url(vtype,venue,year):
    key = '%s/%s/%s%s' % (vtype,venue,venue,year)
    url = 'http://dblp.uni-trier.de/db/%s.html' % key
    return url
url = build_url(vtype,venue,year)
print url

http://dblp.uni-trier.de/db/conf/wsdm/wsdm2012.html


In [4]:
page = requests.get(url)

In [17]:
def get_page_df(pagedict):
    
    vtype = pagedict['vtype']
    venue = pagedict['venue']
    year  = pagedict['year']
    url   = pagedict['url']
    
    page  = pagedict['page']
        
    tree = html.fromstring(page.text)

    headers = tree.xpath("/html/body/div[@id='main']/header[*]")

    df = pd.DataFrame()
    df['Ptitle'] = pd.Series()

    uls = tree.xpath("/html/body/div[@id='main']/ul[*]")[1:]

    ds = []
    for session,ul in zip(headers,uls)[1:]:

        try:
            session_title = session.xpath("h2")[0].text
            session_title = re.sub('\s+',' ',session_title)
        except:
            print 'ERROR session_title',url
            session_title = ''
        
        for li in ul.xpath('li'):
            paper_title = li.xpath("div[@class='data']/span[@class='title']/text()")[0]
            paper_title = re.sub('\s+',' ',paper_title)

            #paper_link = li.xpath("nav[@class='publ']/ul/li[@class='drop-down'][1]/div[@class='body']/ul/li/a/@href")[0]
            #paper_link+= '&preflayout=flat'
            
            paper_key = li.xpath("nav[@class='publ']/ul/li[@class='drop-down'][2]/div[@class='body']/ul[@class='bullets']/li[@class='select-on-click']/small")[0].text

            d = dict(
                Ptitle = paper_title,
                Psession = session_title,
                Ckey = '%s/%s'%(vtype,venue),
                Year = year,
                Pkey = paper_key
            )

            ds.append(d)
    ds

    df = pd.DataFrame(ds)
    df
    return df

#get_page_df(vtype,venue,year)

In [6]:
YEARS = range(2016,1990,-1)
VENUES= 'sigir sigmod sigcomm kdd icml fse mobicom mm'.split()

pages = []

vtype = 'conf'
for year in YEARS:
    for venue in VENUES:
        print year,vtype,venue
        year = str(year)
        
        url = build_url(vtype,venue,year)
        
        try:
            page = requests.get(url)
        except:
            print 'ERROR requests.get(%s)' % url
            continue

        pages.append( dict(
                vtype = vtype,
                venue = venue,
                year  = year,
                url   = url,
                page  = page,
            ) )
        
pages

2016 conf sigir
2016 conf sigmod
2016 conf sigcomm
2016 conf kdd
2016 conf icml
2016 conf fse
2016 conf mobicom
2016 conf mm
2015 conf sigir
2015 conf sigmod
2015 conf sigcomm
2015 conf kdd
2015 conf icml
2015 conf fse
2015 conf mobicom
2015 conf mm
2014 conf sigir
2014 conf sigmod
2014 conf sigcomm
2014 conf kdd
2014 conf icml
2014 conf fse
2014 conf mobicom
2014 conf mm
2013 conf sigir
2013 conf sigmod
2013 conf sigcomm
2013 conf kdd
2013 conf icml
2013 conf fse
2013 conf mobicom
2013 conf mm
2012 conf sigir
2012 conf sigmod
2012 conf sigcomm
2012 conf kdd
2012 conf icml
2012 conf fse
2012 conf mobicom
2012 conf mm
2011 conf sigir
2011 conf sigmod
2011 conf sigcomm
2011 conf kdd
2011 conf icml
2011 conf fse
2011 conf mobicom
2011 conf mm
2010 conf sigir
2010 conf sigmod
2010 conf sigcomm
2010 conf kdd
2010 conf icml
2010 conf fse
2010 conf mobicom
2010 conf mm
2009 conf sigir
2009 conf sigmod
2009 conf sigcomm
2009 conf kdd
2009 conf icml
2009 conf fse
2009 conf mobicom
2009 conf mm


[{'page': <Response [404]>,
  'url': 'http://dblp.uni-trier.de/db/conf/sigir/sigir2016.html',
  'venue': 'sigir',
  'vtype': 'conf',
  'year': '2016'},
 {'page': <Response [404]>,
  'url': 'http://dblp.uni-trier.de/db/conf/sigmod/sigmod2016.html',
  'venue': 'sigmod',
  'vtype': 'conf',
  'year': '2016'},
 {'page': <Response [404]>,
  'url': 'http://dblp.uni-trier.de/db/conf/sigcomm/sigcomm2016.html',
  'venue': 'sigcomm',
  'vtype': 'conf',
  'year': '2016'},
 {'page': <Response [404]>,
  'url': 'http://dblp.uni-trier.de/db/conf/kdd/kdd2016.html',
  'venue': 'kdd',
  'vtype': 'conf',
  'year': '2016'},
 {'page': <Response [404]>,
  'url': 'http://dblp.uni-trier.de/db/conf/icml/icml2016.html',
  'venue': 'icml',
  'vtype': 'conf',
  'year': '2016'},
 {'page': <Response [404]>,
  'url': 'http://dblp.uni-trier.de/db/conf/fse/fse2016.html',
  'venue': 'fse',
  'vtype': 'conf',
  'year': '2016'},
 {'page': <Response [404]>,
  'url': 'http://dblp.uni-trier.de/db/conf/mobicom/mobicom2016.htm

In [18]:
df = pd.DataFrame()

for pagedict in pages:

    #print pagedict
    
    df_ = get_page_df(pagedict)
    
    if df.shape[0] == 0:
        df = df_
    else:
        df = df.append(df_,ignore_index=True)
        
df

ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR session_title http://dblp.uni-trier.de/db/conf/sigmod/sigmod2006.html
ERROR sessio

,Ckey,Pkey,Psession,Ptitle,Year
0,conf/sigir,conf/sigir/Mitra15,Salton Award,Exploring Session Context using Distributed Re...,2015
1,conf/sigir,conf/sigir/EickhoffDT15,Salton Award,An Eye-Tracking Study of Query Reformulation.,2015
2,conf/sigir,conf/sigir/CapraACV15,Salton Award,Differences in the Use of Search Assistance fo...,2015
3,conf/sigir,conf/sigir/OdijkMSR15,Session 1A: Assisting the Search,Dynamic Query Modeling for Related Content Fin...,2015
4,conf/sigir,conf/sigir/McAuleyTSH15,Session 1A: Assisting the Search,Image-Based Recommendations on Styles and Subs...,2015
5,conf/sigir,conf/sigir/PanYLNM15,Session 1A: Assisting the Search,Semi-supervised Hashing with Semantic Confiden...,2015
6,conf/sigir,conf/sigir/YunHER15,Session 1B: Multimedia,Optimal Aggregation Policy for Reducing Tail L...,2015
7,conf/sigir,conf/sigir/LuccheseNOPTV15,Session 1B: Multimedia,QuickScorer: A Fast Algorithm to Rank Document...,2015
8,conf/sigir,conf/sigir/YuM15,Session 1B: Multimedia,High Quality Graph-Based Similarity Search.,2015
9,conf/sigir,conf/sigir/RenR15,Session 1C: Efficient Algorithms,Summarizing Contrastive Themes via Hierarchica...,2015


In [20]:
df

,Ckey,Pkey,Psession,Ptitle,Year
0,conf/sigir,conf/sigir/Mitra15,Salton Award,Exploring Session Context using Distributed Re...,2015
1,conf/sigir,conf/sigir/EickhoffDT15,Salton Award,An Eye-Tracking Study of Query Reformulation.,2015
2,conf/sigir,conf/sigir/CapraACV15,Salton Award,Differences in the Use of Search Assistance fo...,2015
3,conf/sigir,conf/sigir/OdijkMSR15,Session 1A: Assisting the Search,Dynamic Query Modeling for Related Content Fin...,2015
4,conf/sigir,conf/sigir/McAuleyTSH15,Session 1A: Assisting the Search,Image-Based Recommendations on Styles and Subs...,2015
5,conf/sigir,conf/sigir/PanYLNM15,Session 1A: Assisting the Search,Semi-supervised Hashing with Semantic Confiden...,2015
6,conf/sigir,conf/sigir/YunHER15,Session 1B: Multimedia,Optimal Aggregation Policy for Reducing Tail L...,2015
7,conf/sigir,conf/sigir/LuccheseNOPTV15,Session 1B: Multimedia,QuickScorer: A Fast Algorithm to Rank Document...,2015
8,conf/sigir,conf/sigir/YuM15,Session 1B: Multimedia,High Quality Graph-Based Similarity Search.,2015
9,conf/sigir,conf/sigir/RenR15,Session 1C: Efficient Algorithms,Summarizing Contrastive Themes via Hierarchica...,2015


In [24]:
df.to_csv('../data4/dblp.tsv',sep='\t',index=False,encoding='utf8')

In [ ]:
df